In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
import plot_utils
TITLE_FONT = {'fontname': 'Arial', 'fontsize': 12, 'fontweight': 'bold'}
smaller_font = {'fontname': 'Arial', 'fontsize': 10, 'fontweight': 'normal'}
tiny_fontsize = 8
micro_fontsize = 5
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300

In [ ]:
#T related
r = 0.1 # bowl radius
c = 4.2e3 #specific heat capacity of soup
rho = 1e3 # density of soup
l = 0.03 # soup height
h_side = 50 # natural convection coefficient through side
h_top = 50 # natural convection coefficient through top
hob_rate = 7 # hob rate as shown on the hob
# P = 1 # PWM percentage for pot on hob
T_inf = 20 # room temperature

Q_in = 4.88 * 2 ** hob_rate # heat addition rate per unit time


T0 = T_inf

#G related
G_0 = 0.8e-3
alpha_G = 0.8
beta_G = 0.025
gamma_G = 54

#pH related
beta_pH = 0.00585
gamma_pH = 0.9

#n_s related
alpha_t = 10**(-3.5)
k_1 = 0.0001
k_2 = 0.005
k = 1e-6
V_s = 1

def f_G(G_raw, T):
    return G_raw
    c = 0.2
    rate = 0.02    
    return (-c*np.exp(-rate*(T-25))+(c+1)) * G_raw

def simulationStandard(Q, index, salt, vinegar, V_t = 0.4, dt = 0.1):
    alpha_t = 10**(-3.5)
    t = G = pH = n_s = m_s = m_v = 0
    T = T0
    t_total = len(Q) * dt
    counter = 0
    data = []
    salt = salt * 1e-3 # in kg
    vinegar = vinegar * 1e-2 # in kg
    
    
    while t < t_total:
        t += dt
        T += 1/(c*rho*l*math.pi*r**2)*(-(T-T_inf)*(h_top*math.pi*r**2 + 2*h_side*math.pi*r*l) + Q[counter]) * dt
        G_raw = G_0 + alpha_G*m_s + beta_G*m_v + gamma_G*n_s
        G = f_G(G_raw, T)
        n_s += k_1*np.exp(k_2*(T-25))*(alpha_t - n_s*(1/V_t + 1/V_s))
        pH = -np.log10(10e-7 + beta_pH*m_v + gamma_pH*n_s)
        if round(t, 1) in index:
            m_s += float(salt[np.where(index == round(t, 1))])
            m_v += float(vinegar[np.where(index == round(t, 1))])
#             print("m_s, m_v:", m_s, m_v)
        
        counter += 1
        data.append([t, T, G, pH, n_s])

    return np.array(data)

In [ ]:
start = 2

t_point, T_point, G_point, pH_point = plot_utils.analyse_and_plot("Data/0529-2--pH--G--T.npy", False, 4.5, RTD_constant = 0.32)
t_point = t_point[start:]
T_point = T_point[start:]
G_point = G_point[start:]
pH_point = pH_point[start:]

standardData = simulationStandard(Q, index, salt, vinegar, 0.4)
standard_t = standardData[:, 0]
standard_T = standardData[:, 1]
standard_G = standardData[:, 2] * 1000
standard_pH = standardData[:, 3]

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18,4))

ax1.plot(standard_t, standard_T, label = "model prediction")
ax1.plot(t_point, T_point, label = "experiment data")
ax1.set_title("Temperature")
ax1.legend()
ax2.plot(standard_t, standard_G, label = "model prediction")
ax2.plot(t_point, G_point, label = "experiment data")
ax2.set_title("Conductance")
ax2.legend()
ax3.plot(standard_t, standard_pH, label = "model prediction")
ax3.plot(t_point, pH_point, label = "experiment data")
ax3.set_title("pH")
ax3.legend()

In [ ]:
plt.rcParams["figure.figsize"] = (2.95, 2)

plotwidth = 0.8
barwidth = 30

t = np.linspace(0, 800, 8000)
Q = np.array([1]*4500 + [0.25]*3500) * 4.88*2**7
index = np.array([300, 400, 500])
salt = np.array([1, 1, 1])
vinegar = np.array([1, 1, 1])

# Create the figure and left y-axis
fig, ax1 = plt.subplots()

# Plot the first dataset on the left y-axis
ax1.plot(t, Q, label = "heat input", color = "Blue", linewidth = plotwidth)
ax1.set_xlabel("time(t)", fontsize = tiny_fontsize)
ax1.set_ylabel(r'$\dot{Q_{in}}$ (W)', color='Black', fontsize = tiny_fontsize)
ax1.set_ylim([0, 1.15 * np.max(Q)])
ax1.tick_params('x', colors='Black', labelsize = tiny_fontsize)
ax1.tick_params('y', colors='Black', labelsize = tiny_fontsize)

# Create the right y-axis
ax2 = ax1.twinx()

# Plot the second dataset on the right y-axis
ax2.bar(index, salt, barwidth, label = "salt_input", color = "Red")
ax2.bar(index, vinegar, barwidth, label = "vinegar_input", bottom = salt, color = "Green")

ax2.set_ylabel('unit of salt/vinegar added', color='Black', fontsize = tiny_fontsize)
ax2.yaxis.set_label_coords(1.11, 0.5)
ax2.tick_params('y', colors='Black', labelsize = tiny_fontsize)
ax2.set_ylim([0, 8.5])

# Add legends and display the plot
ax1.legend(loc='upper left', fontsize = tiny_fontsize - 2)
ax2.legend(loc='upper right', fontsize = tiny_fontsize - 2)
# plt.title("Cooking Input")
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = (2.95, 2)
plt.plot(standard_t, standard_T, label = "human model", linewidth = plotwidth, linestyle='--', color = "Blue")
plt.plot(t_point, T_point, label = "human experiment", linewidth = plotwidth, color = "Blue")
plt.tick_params(axis='both', labelsize=tiny_fontsize)
plt.xlabel("time(t)", fontsize = tiny_fontsize)
plt.ylabel("T (°C)", fontsize = tiny_fontsize, labelpad = -5)
plt.legend(fontsize = tiny_fontsize - 2)

In [ ]:
plt.plot(standard_t, standard_G, label = "human model", linewidth = plotwidth, linestyle='--', color = "Blue")
plt.plot(t_point, G_point, label = "human experiment", linewidth = plotwidth, color = "Blue")
plt.tick_params(axis='both', labelsize=tiny_fontsize)
plt.xlabel("time(t)", fontsize = tiny_fontsize)
plt.ylabel("G (mS)", fontsize = tiny_fontsize)
plt.legend(fontsize = tiny_fontsize - 2)

In [ ]:
plt.plot(standard_t, standard_pH, label = "human model", linewidth = plotwidth, linestyle='--', color = "Blue")
plt.plot(t_point, pH_point, label = "human experiment", linewidth = plotwidth, color = "Blue")
plt.tick_params(axis='both', labelsize=tiny_fontsize)
plt.xlabel("time(t)", fontsize = tiny_fontsize)
plt.ylabel("pH", fontsize = tiny_fontsize)
plt.legend(fontsize = tiny_fontsize - 2)

In [ ]:
#experiment

start = 2
time_shift = 50
t_point, T_point, G_point, pH_point = plot_utils.analyse_and_plot("Data/0529-2--pH--G--T.npy", False, 4.5, RTD_constant = 0.32)
t_point = t_point + time_shift
G_point = G_point * 0.9
t_point = t_point[start:]
T_point = T_point[start:]
G_point = G_point[start:]
pH_point = pH_point[start:]

#MPC control

start = 21
end = 2
t_point_MPC, T_point_MPC, G_point_MPC, pH_point_MPC = plot_utils.analyse_and_plot("Data/0529-6--pH--G--T.npy", False, 4.5, RTD_constant = 0.32)
t_point_MPC = t_point_MPC[start:-end]
T_point_MPC = T_point_MPC[start:-end]
G_point_MPC = G_point_MPC[start:-end]
pH_point_MPC = pH_point_MPC[start:-end]

#model

t = np.linspace(0, 900, 9000)
Q = np.array([1]*4300) * 4.88*2**7
index = np.array([10])
salt = np.array([3.5])
vinegar = np.array([3.6])

standardData = simulationStandard(Q, index, salt, vinegar, 0.4)
model_t = standardData[:, 0]
model_T = standardData[:, 1]
model_G = standardData[:, 2] * 1000
model_pH = standardData[:, 3]

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18,4))

ax1.plot(t_point, T_point, label = "manual")
ax1.plot([0, t_point[-1]], [T_point[-1], T_point[-1]], label = "end state")
ax1.plot(t_point_MPC, T_point_MPC, label = "MPC")
ax1.plot(model_t, model_T, label = "MPC model")
ax1.set_title("Temperature")
ax1.legend()
ax2.plot(t_point, G_point, label = "manual")
ax2.plot([0, t_point[-1]], [G_point[-1], G_point[-1]], label = "end state")
ax2.plot(t_point_MPC, G_point_MPC, label = "MPC")
ax2.plot(model_t, model_G, label = "MPC model")
ax2.set_title("Conductance")
ax2.legend()
ax3.plot(t_point, pH_point, label = "manual")
ax3.plot([0, t_point[-1]], [pH_point[-1], pH_point[-1]], label = "end state")
ax3.plot(t_point_MPC, pH_point_MPC, label = "MPC")
ax3.plot(model_t, model_pH, label = "MPC model")
ax3.set_title("pH")
ax3.legend()

In [ ]:
plt.rcParams["figure.figsize"] = (2.95, 2)

plotwidth = 0.8
barwidth = 15

t = np.linspace(0, 430, 4300)
Q = np.array([1]*4300) * 4.88*2**7
index = np.array([10])
salt = np.array([3.5])
vinegar = np.array([3.6])

# Create the figure and left y-axis
fig, ax1 = plt.subplots()

# Plot the first dataset on the left y-axis
ax1.plot(t, Q, label = "heat input", color = "Blue", linewidth = plotwidth)
ax1.set_xlabel("time(t)", fontsize = tiny_fontsize)
ax1.set_ylabel(r'$\dot{Q_{in}}$ (W)', color='Black', fontsize = tiny_fontsize)
ax1.set_ylim([0, 1.15 * np.max(Q)])
ax1.tick_params('x', colors='Black', labelsize = tiny_fontsize)
ax1.tick_params('y', colors='Black', labelsize = tiny_fontsize)

# Create the right y-axis
ax2 = ax1.twinx()

# Plot the second dataset on the right y-axis
ax2.bar(index, salt, barwidth, label = "salt_input", color = "Red")
ax2.bar(index, vinegar, barwidth, label = "vinegar_input", bottom = salt, color = "Green")

ax2.set_ylabel('unit of salt/vinegar added', color='Black', fontsize = tiny_fontsize)
ax2.yaxis.set_label_coords(1.11, 0.5)
ax2.tick_params('y', colors='Black', labelsize = tiny_fontsize)
ax2.set_ylim([0, 8.5])

# Add legends and display the plot
ax1.legend(loc='upper left', fontsize = tiny_fontsize - 2)
ax2.legend(loc='upper right', fontsize = tiny_fontsize - 2)
# plt.title("Cooking Input")
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = (2.95, 2)
plt.plot(t_point, T_point, label = "human experiment", linewidth = plotwidth, color = "Blue")
plt.plot([0, t_point[-1]], [T_point[-1], T_point[-1]], label = "end state", linestyle='--', linewidth = plotwidth, color = "Green")
plt.plot(model_t, model_T, label = "MPC model", linestyle='--', color = "Orange", linewidth = plotwidth)
plt.plot(t_point_MPC, T_point_MPC, label = "MPC experiment", color = "Orange", linewidth = plotwidth)
plt.xlabel("time(t)", fontsize = tiny_fontsize)
plt.ylabel("T (°C)", fontsize = tiny_fontsize, labelpad = -5)
plt.legend(fontsize = tiny_fontsize - 2)

In [ ]:
plt.plot(t_point, G_point, label = "human experiment", linewidth = plotwidth, color = "Blue")
plt.plot([0, t_point[-1]], [G_point[-1], G_point[-1]], label = "end state", linestyle='--', linewidth = plotwidth, color = "Green")
plt.plot(model_t, model_G, label = "MPC model", linestyle='--', color = "Orange", linewidth = plotwidth)
plt.plot(t_point_MPC, G_point_MPC, label = "MPC experiment", color = "Orange", linewidth = plotwidth)
plt.xlabel("time(t)", fontsize = tiny_fontsize)
plt.ylabel("G (mS)", fontsize = tiny_fontsize)
plt.legend(fontsize = tiny_fontsize - 2)

In [ ]:
plt.plot(t_point, pH_point, label = "human experiment", linewidth = plotwidth, color = "Blue")
plt.plot([0, t_point[-1]], [pH_point[-1], pH_point[-1]], label = "end state", linestyle='--', linewidth = plotwidth, color = "Green")
plt.plot(model_t, model_pH, label = "MPC model", linestyle='--', color = "Orange", linewidth = plotwidth)
plt.plot(t_point_MPC, pH_point_MPC, label = "MPC experiment", color = "Orange", linewidth = plotwidth)
plt.xlabel("time(t)", fontsize = tiny_fontsize)
plt.ylabel("pH", fontsize = tiny_fontsize)
plt.legend(fontsize = tiny_fontsize - 2)

In [ ]:
#experiment
t = np.linspace(0, 800, 8000)
Q = np.array([1]*4500 + [0.25]*3500) * 4.88*2**7
index = np.array([300, 400, 500])
salt = np.array([1, 1, 1])
vinegar = np.array([1, 1, 1])

standardData = simulationStandard(Q, index, salt, vinegar, 0.4)
standard_t = standardData[:, 0]
standard_T = standardData[:, 1]
standard_G = standardData[:, 2] * 1000
standard_pH = standardData[:, 3]


#PID control

start = 1
end = 3
t_point_PID, T_point_PID, G_point_PID, pH_point_PID = plot_utils.analyse_and_plot("Data/0529-7--pH--G--T.npy", False, 4.5, RTD_constant = 0.32)
t_point_PID = t_point_PID[start:-end]
T_point_PID = T_point_PID[start:-end]
G_point_PID = G_point_PID[start:-end]
pH_point_PID = pH_point_PID[start:-end]

#model

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18,4))

ax1.plot(standard_t, standard_T, label = "manual model")
ax1.plot([0, t_point[-1]], [T_point[-1], T_point[-1]], label = "end state")
ax1.plot(t_point_PID, T_point_PID, label = "PID")
ax1.set_title("Temperature")
ax1.legend()
ax2.plot(standard_t, standard_G, label = "manual model")
ax2.plot([0, t_point[-1]], [G_point[-1], G_point[-1]], label = "end state")
ax2.plot(t_point_PID, G_point_PID, label = "PID")
ax2.set_title("Conductance")
ax2.legend()
ax3.plot(standard_t, standard_pH, label = "manual model")
ax3.plot([0, t_point[-1]], [pH_point[-1], pH_point[-1]], label = "end state")
ax3.plot(t_point_PID, pH_point_PID, label = "PID")
ax3.set_title("pH")
ax3.legend()

In [ ]:
plt.rcParams["figure.figsize"] = (2.95, 2)

plotwidth = 0.8
barwidth = 30

t = np.linspace(0, 800, 8000)

Q_chip = [0.26, 0.37, 0.47, 0.24, 0.26, 0.17, 0.34, 0.32, 0.21, 0.67, 0.76, 0.65]
Q = 4400*[1]

for n in Q_chip:
    Q += [1]*int(n*300) + [0]*(300-int(n*300))

Q = np.array(Q)*4.88*2**7
index = np.array([300, 360, 420, 480, 540, 600, 660, 720])
salt = np.array([0.12, 0.23, 0.325, 0.2, 0.11, 0.07, 0.14, 0.22])
vinegar = np.array([0.41, 0.32, 0.34, 0.2, 0.13, 0.12, 0.06, 0.03])

# Create the figure and left y-axis
fig, ax1 = plt.subplots()

# Plot the first dataset on the left y-axis
ax1.plot(t, Q, label = "heat input", color = "Blue", linewidth = plotwidth)
ax1.set_xlabel("time(t)", fontsize = tiny_fontsize)
ax1.set_ylabel(r'$\dot{Q_{in}}$ (W)', color='Black', fontsize = tiny_fontsize)
ax1.set_ylim([0, 1.15 * np.max(Q)])
ax1.tick_params('x', colors='Black', labelsize = tiny_fontsize)
ax1.tick_params('y', colors='Black', labelsize = tiny_fontsize)

# Create the right y-axis
ax2 = ax1.twinx()

# Plot the second dataset on the right y-axis
ax2.bar(index, salt, barwidth, label = "salt_input", color = "Red")
ax2.bar(index, vinegar, barwidth, label = "vinegar_input", bottom = salt, color = "Green")

ax2.set_ylabel('unit of salt/vinegar added', color='Black', fontsize = tiny_fontsize)
ax2.yaxis.set_label_coords(1.11, 0.5)
ax2.tick_params('y', colors='Black', labelsize = tiny_fontsize)
ax2.set_ylim([0, 8.5])

# Add legends and display the plot
ax1.legend(loc='upper left', fontsize = tiny_fontsize - 2)
ax2.legend(loc='upper right', fontsize = tiny_fontsize - 2)
# plt.title("Cooking Input")
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = (2.95, 2)
plt.plot(standard_t, standard_T, label = "human model", linestyle='--', linewidth = plotwidth, color = "Blue")
plt.plot([0, t_point[-1]], [T_point[-1], T_point[-1]], label = "end state", linestyle='--', linewidth = plotwidth, color = "Green")
plt.plot(t_point_PID, T_point_PID, label = "PID experiment", color = "Orange", linewidth = plotwidth)
plt.xlabel("time(t)", fontsize = tiny_fontsize)
plt.ylabel("T (°C)", fontsize = tiny_fontsize, labelpad = -5)
plt.legend(fontsize = tiny_fontsize - 2)

In [ ]:
plt.plot(standard_t, standard_G, label = "human model", linestyle='--', linewidth = plotwidth, color = "Blue")
plt.plot([0, t_point[-1]], [G_point[-1], G_point[-1]], label = "end state", linestyle='--', linewidth = plotwidth, color = "Green")
plt.plot(t_point_PID, G_point_PID, label = "PID experiment", color = "Orange", linewidth = plotwidth)
plt.xlabel("time(t)", fontsize = tiny_fontsize)
plt.ylabel("G (mS)", fontsize = tiny_fontsize)
plt.legend(fontsize = tiny_fontsize - 2)


# plt.plot(t_point, G_point, label = "manual", linewidth = plotwidth, color = "Blue")
# plt.plot([0, t_point[-1]], [G_point[-1], G_point[-1]], label = "end state", linestyle='--', linewidth = plotwidth, color = "Green")
# plt.plot(model_t, model_G, label = "MPC model", linestyle='--', color = "Orange", linewidth = plotwidth)
# plt.plot(t_point_MPC, G_point_MPC, label = "MPC experiment", color = "Orange", linewidth = plotwidth)
# plt.xlabel("time(t)", fontsize = tiny_fontsize)
# plt.ylabel("G (mS)", fontsize = tiny_fontsize)
# plt.legend(fontsize = tiny_fontsize - 2)

In [ ]:
plt.plot(standard_t, standard_pH, label = "human model", linestyle='--', linewidth = plotwidth, color = "Blue")
plt.plot([0, t_point[-1]], [pH_point[-1], pH_point[-1]], label = "end state", linestyle='--', linewidth = plotwidth, color = "Green")
plt.plot(t_point_PID, pH_point_PID, label = "PID experiment", color = "Orange", linewidth = plotwidth)
plt.xlabel("time(t)", fontsize = tiny_fontsize)
plt.ylabel("pH", fontsize = tiny_fontsize)
plt.legend(fontsize = tiny_fontsize - 2)